# 🤖 Formation RAG – Notebook intégral

---
## 🎯 Objectifs pédagogiques

Cette formation a pour but de vous initier au concept de **RAG (Retrieval-Augmented Generation)**. À la fin de cette formation, vous serez capable de :

- Comprendre les composants essentiels d’un système RAG
- Manipuler du code Python sur Colab
- Installer et utiliser **Ollama** pour faire tourner un modèle LLM localement
- Tester un prototype sur un document PDF
- Explorer les étapes vers une mise en production (abordées en deuxième partie)


# 🚧 Séquence 1.0 – Installation d’Ollama & des modèles

## 🛠️ Installation d’Ollama (Linux / Colab)

**Ollama** est un outil qui permet d'exécuter des modèles de langage (LLM) **en local**. Il installe un serveur local qui héberge les modèles et facilite leur utilisation.

Utiliser des modèles localement présente plusieurs avantages :
- **Confidentialité** : les données ne sortent pas de votre machine
- **Coût** : pas besoin de serveur distant ou d’API payante
- **Performance** : temps de réponse plus rapide dans certains cas

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## 🚀 Lancement Ollama en arrière‑plan


Cette cellule démarre le **serveur Ollama**, c’est-à-dire le processus qui permettra de faire fonctionner les modèles en local.

### 💡 Pourquoi c’est important ?  
Les modèles ne peuvent répondre à vos requêtes que si un serveur tourne en arrière-plan. Ce serveur **écoute** vos demandes et renvoie les réponses générées.


In [ ]:
import subprocess, time
ollama_proc = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)
print('✅ Ollama est prêt')

✅ Ollama est prêt


## 📥 Téléchargement des modèles

Ici, on télécharge deux modèles nécessaires pour la suite :

- **Llama 3.2 (3B)** : un modèle de génération de texte développé en open source par Meta
- **Nomic Embed Text** : un modèle spécialisé pour convertir du texte en vecteurs numériques (embeddings), utilisé plus tard dans la partie RAG

Ces modèles sont stockés localement pour être utilisés sans connexion externe.

In [ ]:
!ollama pull llama3.2:3B
!ollama pull nomic-embed-text:latest

# 🔗 Séquence 1.1 – Bootstrap Colab

## 🔌 Connexion à Google Drive

Cette cellule permet de connecter votre Google Drive à l’environnement Colab.

### 💡 Pourquoi ?  
Cela vous permet d'accéder facilement à vos fichiers (PDF, datasets, etc.) depuis le Notebook.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 📥 Clone du dépôt

On télécharge ici le dossier de formation depuis GitHub.

### 💡 Pourquoi ?  
GitHub est une plateforme de partage de code. Le dépôt contient tous les **fichiers utiles à la formation**, comme les scripts, les données ou les modèles nécessaires.




In [ ]:
%cd /content/gdrive/MyDrive
!git clone https://github.com/antoinecstl/FormationEYAI.git
%cd FormationEYAI

/content/gdrive/MyDrive
fatal: destination path 'FormationEYAI' already exists and is not an empty directory.
/content/gdrive/MyDrive/FormationEYAI


## 🛠️ Installation des dépendances

Cette cellule installe toutes les **bibliothèques Python** nécessaires à l'exécution du notebook, à partir du fichier `requirements.txt`.


### 💡 Pourquoi on fait ça ?
Plutôt que d’installer chaque outil un par un, ce fichier centralise tout, ce qui fait gagner du temps et évite les erreurs d’oubli ou d’incompatibilité.

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


# 🔍 Séquence 1.2 – Bases du RAG : embeddings & similarité

## 🧠 Création d'embeddings avec un modèle local

Pour pouvoir comparer des phrases ou retrouver des documents pertinents, on doit **transformer du texte en vecteurs numériques** (embeddings). Ces vecteurs capturent le sens des mots ou des phrases dans un espace mathématique.

### 🔧 Que fait cette cellule ?
- Elle définit une fonction `embed_texts` qui prend une **liste de phrases** en entrée.
- Chaque phrase est transformée en vecteur via le modèle `nomic-embed-text` installé localement avec Ollama.
- Elle retourne un tableau `numpy` contenant les vecteurs (`shape = (n, d)`), où :
  - `n` est le nombre de phrases
  - `d` est la dimension de l’espace d’embedding

Ces vecteurs seront utiles pour calculer des similarités ou faire de la recherche sémantique.


In [ ]:
import numpy as np, ollama

EMBED_MODEL = "nomic-embed-text:latest"

def embed_texts(texts):
    """Retourne un np.ndarray shape (n, d)"""
    return np.array([ollama.embeddings(model=EMBED_MODEL, prompt=t)['embedding'] for t in texts], dtype='float32')

## 🔬 Test d'embedding : comparaison de phrases

Ici, on mesure la **similarité** entre deux phrases à l’aide de leurs embeddings.

#### 🔍 Que fait cette cellule ?
- Elle convertit chaque phrase en vecteur (embedding)
- Elle mesure leur proximité à l’aide d’un **produit scalaire**
- Le score obtenu indique le **niveau de similarité sémantique**

#### 📊 Comment lire le score ?
- `1` : phrases très proches (sens similaire)
- `0` : phrases sans lien
- `-1` : phrases opposées

✏️ Vous pouvez modifier les phrases pour tester différents cas.

In [ ]:
phrase1 = "The cat is sleeping on the sofa"
phrase2 = "cat is not sleeping"

vecs = embed_texts([phrase1, phrase2])
sim = float(vecs[0] @ vecs[1] / (np.linalg.norm(vecs[0])*np.linalg.norm(vecs[1])))
print(f"Similarité : {sim:.3f}")

Similarité : 0.796


# 📐 Séquence 1.3 – Chunking & nettoyage d’un PDF d’exemple

SLIDE sur qu'est-ce qu'un chunk et comment c'est utilisé, le découpage est intelligent.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def auto_chunk_size(tok:int)->int:
    return 1024 if tok<8000 else 768 if tok<20000 else 512

def chunk_document(text:str):
    size=auto_chunk_size(len(text.split()))
    splitter=RecursiveCharacterTextSplitter(
        separators=["\n\n","\n",". "],
        chunk_size=size,
        chunk_overlap=size//4,
        length_function=len,
        )
    return [c for c in splitter.split_text(text) if len(c)>100]

## 📖 Chargement du PDF d'exemple

On commence par **extraire le texte brut du PDF** page par page grâce à la librairie PyPDF2.

### 💡 Pourquoi faire ça ?  
- Cela permet de récupérer tout le contenu textuel du document.  
- On peut ensuite estimer sa taille en nombre de mots (tokens), ce qui aide à adapter les traitements (chunking, embeddings, etc.).

In [ ]:
from PyPDF2 import PdfReader

sample_path = "/content/FormationEYAI/rapport.pdf"  # fourni dans le repo
pages = PdfReader(sample_path).pages
full_text = "\n".join(p.extract_text() or "" for p in pages)

print(f"📄 Le document contient {len(full_text.split())} tokens environ.")

📄 Le document contient 74 tokens environ.


### 🌳 Chunking du PDF
Ici, le text extrait du pdf est découpé en plus petit segments (chunks), afin de préparer le texte pour de l'indexation.


### Pourquoi créer des chunks ?  
- Les modèles ne peuvent pas traiter de très longs textes d’un coup.  
- Le chunking permet de diviser le contenu en morceaux cohérents et exploitables.  
- On peut ensuite traiter chaque chunk indépendamment (calcul d’embeddings, recherche, etc.).


✏️ Parcourer la lise de chunks générés afin de valider le bon découpage du contenu.

In [ ]:
chunks = chunk_document(full_text)
print(f"🌳 {len(chunks)} chunks créés. Aperçu :\n{chunks[0][:300]}…")

🌳 1 chunks créés. Aperçu :
Les plages sont des lieux naturels magnifiques où la terre rencontre la mer. Elles offrent
un  cadre  paisible  propice  à  la  détente,  aux  promenades  au  bord  de  l’eau  et  aux
activités  nautiques.  Le  sable  chaud,  le  bruit  apaisant  des  vagues  et  l’horizon  infini
créent une atmosph…


# 📊 Séquence 1.4 – Index vectoriel FAISS


Les vecteurs obtenus à partir des chunks sont rangés dans une structure appelée **index FAISS**.

### Qu’est-ce que FAISS ?  
- Un outil très rapide pour rechercher les vecteurs proches dans un grand ensemble.  
- Permet de retrouver rapidement les documents les plus similaires à une requête.

### 💡 Pourquoi créer cet index ?  
- Pour accélérer les recherches dans la base de documents vectorisés.  
- C’est indispensable dès qu’on a beaucoup de données à parcourir.

In [ ]:
import faiss, numpy as np

def build_faiss_index(vectors:np.ndarray)->faiss.IndexFlatIP:
    d=vectors.shape[1]
    idx=faiss.IndexFlatIP(d)
    idx.add(vectors.astype('float32'))
    return idx

## 🧪 Construction index chunks


Chaque chunk est transformé en vecteur numérique (embedding), puis ajouté à l’index FAISS.

### Ce que ça signifie :  
- On passe de textes à vecteurs.  
- On construit une base efficace pour retrouver les chunks les plus pertinents rapidement.

In [ ]:
import numpy as np

chunk_vecs = embed_texts(chunks)
index = build_faiss_index(chunk_vecs)

print(index.ntotal, "Nombre de vecteurs total dans l'index")

1 Nombre de vecteurs total dans l'index


# 🧮 Séquence 1.5 – Algorithme MMR

MMR permet de sélectionner des passages à la fois **pertinents** et **diversifiés** pour une requête donnée.

### Pourquoi c’est important ?  
- Sélectionner uniquement les passages les plus similaires peut donner des résultats redondants.  
- MMR équilibre la similarité à la question et la diversité entre passages sélectionnés.

Cet algorithme améliore la qualité des résultats en évitant les répétitions.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr(query_vec:np.ndarray, cand:np.ndarray, k:int=5, λ:float=0.3):
    selected, rest = [], list(range(len(cand)))
    while len(selected)<min(k,len(rest)):
        best, best_score = None, -1e9
        for idx in rest:
            sim_q = float(query_vec @ cand[idx]/(np.linalg.norm(query_vec)*np.linalg.norm(cand[idx])+1e-6))
            sim_s = max(cosine_similarity(cand[idx][None,:], cand[selected])[0]) if selected else 0.
            score = λ*sim_q - (1-λ)*sim_s
            if score>best_score:
                best, best_score = idx, score
        selected.append(best); rest.remove(best)
    return selected

## 🔬 Test MMR
Ici, l'algorithme MMR est appelé pour trouver les 3 passages les plus pertinents et variés en réponse à la question.

✏️Modifier la question pour s'assurer que MMR sélectionne bien des passages différents mais liés à la question.

In [ ]:
import numpy as np

q_vec = embed_texts(["Sujet principal du rapport ?"])[0]
sel = mmr(q_vec, chunk_vecs, 3)
print(sel)

[0]


# 🧑‍🎤 Séquence 1.6 – Prompt engineering

## 🧠 Construction du prompt

Après avoir sélectionné les passages du document les plus pertinents et variés par rapport à la question,  
on construit un prompt clair et structuré avec `build_prompt`.

`ctxs` représente les informations du document jugées pertinentes à la `question` posée.

### 💡 Pourquoi on fait ça ?  
Cette étape est essentielle pour que le modèle fournisse une réponse ciblée et fiable,  
en s’appuyant uniquement sur les données extraites du document.

In [ ]:
def build_prompt(question:str, ctxs:list[str]):
    ctx_block="\n\n".join(f"[{i+1}] {c}" for i,c in enumerate(ctxs))
    system="Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. Citez les sources [n]."
    return [
        {"role":"system","content":system},
        {"role":"user","content":f"CONTEXTE(S):\n{ctx_block}\n\nQUESTION: {question}\n\nRéponse:"}
    ]


A supprimer

In [ ]:
print(build_prompt("Pourquoi le ciel est bleu ?", ["La diffusion Rayleigh explique la couleur du ciel."]))

[{'role': 'system', 'content': 'Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. Citez les sources [n].'}, {'role': 'user', 'content': 'CONTEXTE(S):\n[1] La diffusion Rayleigh explique la couleur du ciel.\n\nQUESTION: Pourquoi le ciel est bleu\xa0?\n\nRéponse:'}]


## 🧑‍🎤 Premier Appel au LLM

Ce code envoie une liste de messages au modèle `llama3.2:3B` via la fonction `_call_llm`.

- Le premier message définit le rôle ou le comportement attendu du modèle.  
- Le second contient la question posée.

On peut ajuster la créativité (`temperature`) et la longueur de la réponse (`max_tokens`).

✏️ Modifie ces paramètres et la question pour tester et comprendre l’impact sur les réponses.


In [ ]:
from typing import List, Dict

MODEL_NAME = "llama3.2:3B"

def _call_llm(messages: List[Dict[str, str]], *, temperature: float = 0.1, max_tokens: int = 2048, stream: bool = False):
    """Enveloppe simple autour de ollama.chat pour usage direct."""
    return ollama.chat(
        model=MODEL_NAME,
        messages=messages,
        stream=stream,
        options={"temperature": temperature, "num_predict": max_tokens},
    )

# 🧪 Exemple d'appel
messages = [
    {"role": "system", "content": "Tu es un assistant concis"},
    {"role": "user", "content": "Donne-moi la capitale de l’Italie"}
]
print(_call_llm(messages)["message"]["content"].strip())

La capitale de l'Italie est Rome.


# 🔗 Séquence 1.8 – Assemblage mini‑RAG (prototype)

Ici, on combine toutes les étapes vues précédemment pour créer un système simple de RAG qui répond à une question à partir d’un document.

1. Transformer la question en vecteur (embedding) avec `embed_texts`.  
2. Chercher les passages les plus proches dans l’index FAISS (`index.search`).  
3. Récupérer les textes correspondants à ces passages.  
4. Construire un prompt structuré avec `build_prompt`.  
5. Appeler le modèle de langage avec `_call_llm` pour générer la réponse.  
6. Retourner la réponse et les passages utilisés.

Cete fonction montre comment utiliser les embeddings et l’indexation pour alimenter un LLM en contexte précis.

In [ ]:
def ask(question: str, chunks: List[str], vecs: np.ndarray, top_k: int = 3):
    # Recherche des chunks pertinents
    q_vec = embed_texts([question])[0]
    _, I = index.search(q_vec[None, :], top_k)
    ctx = [chunks[i] for i in I[0]]
    # Préparation du prompt
    prompt = build_prompt(question, ctx)
    # Appel LLM et retour de la réponse
    answer = _call_llm(prompt)["message"]["content"].strip()
    return answer, I[0]

## 🧪 Prototype RAG sur le PDF


Maintenant, on peut poser une question sur le PDF `rapport.pdf` et obtenir une réponse sourcée basée sur le contenu réel du document.

✏️ Change la question pour explorer différentes réponses !



In [ ]:
question = "Quel est le thème principal de ce document ?"
print(ask(question, chunks, chunk_vecs))

NameError: name 'ask' is not defined

---
# 🛠️ Partie 2 – Mise en production avec `app_finale.py`

## 🏗️ Séquence 2.1 – Préparer l’environnement

In [ ]:
# @title ⚙️ Optionnel : créer un virtualenv local
# !python -m venv venv && source venv/bin/activate

## 📦 Séquence 2.2 – Récupérer `app_finale.py`

In [ ]:
# @title 📥 Copier le script final
%cp FormationEYAI/app_finale.py ./app_finale.py
!ls -l app_finale.py

## 🤖 Séquence 2.3 – Démarrer Ollama en production

In [ ]:
# @title 🚀 Lancement Ollama en arrière‑plan
import subprocess, time, os, signal
ollama_proc = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)
print('✅ Ollama est prêt')

## 🖥️ Séquence 2.4 – Lancer l’application Streamlit

In [ ]:
# @title 🎛️ Run Streamlit + LocalTunnel
!pip install -q streamlit localtunnel
!streamlit run app_finale.py &>/content/logs.txt & npx localtunnel --port 8501

## 📈 Séquence 2.5 – Observabilité

- Temps de réponse, logs Streamlit (`tail -f content/logs.txt`)  
- Sécurité des données
- Coût GPU / CPU


## 🎓 Mini‑projet final

Ajoutez un **second modèle d’embedding** ou la prise en charge d’un format `.docx`, puis présentez vos résultats.
